### Import Data

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
a = []
while(1):
    a.append(1)

In [0]:
# tx = pd.read_pickle(path+"GS_dataframe_0_to_49.pickle")
# (tx.set_index(np.arange(1000000, 1000050, 1))).to_pickle(path+"GS_dataframe_0_to_49.pickle")

In [0]:
import os
import numpy as np
import pandas as pd

path = "/content/drive/My Drive/IR End Review/dataframe/"

data = pd.DataFrame()

for i in os.listdir(path):
    df = pd.read_pickle(path+i)
    df['temp_col_len'] = df.full.str.len()
    print(i)
    data = pd.concat([data, df[df['temp_col_len']>1].drop(columns = ['temp_col_len'])], axis = 0, ignore_index=True)
data

In [0]:
GS_index = (np.array(data.index))[-50:]
GS_index

Split the body of Text into Sentences

In [0]:
# Sentence tokenization
# and tagging of document number t sentences

temp = data.copy()

set = temp.index
# set = np.arange(0, 51, 1)
# set = [0, 1, 2]

# temp = data

full_sent = pd.DataFrame()
summ_sent = pd.DataFrame()

full_docs = []
summ_docs = []
GS_docs = []

ori_sent = pd.DataFrame()
GS_sent = pd.DataFrame()
GS_ori_sent = pd.DataFrame()

for i in set:
  if i not in GS_index:
    full = temp['full'].iloc[i]
    summary = temp['summary'].iloc[i]
    ori = temp['full_orignal'].iloc[i]
    
#     print(full)
#     break
    v = full.split("*")    
    t1 = pd.DataFrame(v)
    
    v = summary.split('*')
    t2 = pd.DataFrame(v)
    
    v = ori.split('*')
    t5 = pd.DataFrame(v)
    
    t3 = np.linspace(i, i, num = len(t1))
    t4 = np.linspace(i, i, num = len(t2))

    # print(t1)
    
    if(len(t1)==len(t5)):
        full_docs = np.append(full_docs, t3)
        summ_docs = np.append(summ_docs, t4)
        full_sent = pd.concat([full_sent, t1], ignore_index=True)
        summ_sent = pd.concat([summ_sent, t2], ignore_index=True)
        ori_sent = pd.concat([ori_sent, t5], ignore_index=True)
    else:
        print("Doc No. : ", i,"has a problem")
  
  else:
    full = temp['full'].iloc[i]
    v = full.split("*")    
    t1 = pd.DataFrame(v)

    ori = temp['full_orignal'].iloc[i]
    v = ori.split('*')
    t5 = pd.DataFrame(v)

    t3 = np.linspace(i, i, num = len(t1))

    if(len(t1)==len(t5)):
        GS_docs = np.append(GS_docs, t3)
        GS_sent = pd.concat([GS_sent, t1], ignore_index=True)
        GS_ori_sent = pd.concat([GS_ori_sent, t5], ignore_index=True)
    else:
      print("GS Doc No. : ", i-1060,"has a problem")

print('summary sentence tokenization')
print(len(summ_docs))
print(len(summ_sent))
print(len(full_docs))
print(len(full_sent))
print(len(ori_sent))

print(len(GS_docs))
print(len(GS_sent))
print(len(GS_ori_sent))

# summ_sent_mod = pd.DataFrame(summ_sent, columns=['Summary_Sentences'])
summ_sent_mod = summ_sent.set_axis(['Sentences'], axis=1, inplace=False)
# summ_sent_mod

full_sent_mod = full_sent.set_axis(['Sentences'], axis=1, inplace=False)
# full_sent_mod

In [0]:
# combining summary and full-text sentences for tfidf vectorization of sentences

# -----------------------------------------------------------------------------------------------------------------------------

GS_sent_mod = GS_sent.set_axis(['Sentences'], axis=1, inplace=False)

tmp1 = pd.DataFrame(summ_sent_mod)
tmp2 = pd.DataFrame(full_sent_mod)
tmp3 = pd.DataFrame(GS_sent_mod)
combined = pd.concat([tmp1, tmp2], axis = 0, ignore_index=True, sort = False)
combined = pd.concat([combined, tmp3], axis = 0, ignore_index=True, sort = False)
combined.columns = ['sents']

# -----------------------------------------------------------------------------------------------------------------------------

print(combined.shape)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer as tuttu
from nltk.tokenize import RegexpTokenizer
import scipy

# -----------------------------------------------------------------------------------------------------------------------------

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf_func = tuttu(ngram_range = (1, 1), tokenizer = token.tokenize, lowercase = True)
tfidf = tfidf_func.fit_transform(combined['sents'])

# -----------------------------------------------------------------------------------------------------------------------------

tfidf.shape

In [0]:
# dimension reduction
from sklearn.decomposition import PCA, TruncatedSVD

svd = TruncatedSVD(100)
a = svd.fit_transform(tfidf[:,:])

del tfidf

print(a.shape)
print(a)

In [0]:
# ## Function to reduce the DF size
# def reduce_mem_usage(df, verbose=True):
#     numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#     start_mem = df.memory_usage().sum() / 1024**2    
#     for col in df.columns:
#         col_type = df[col].dtypes
#         if col_type in numerics:
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                 df[col] = df[col].astype(np.float16)
#             elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                 df[col] = df[col].astype(np.float32)
#             else:
#                 df[col] = df[col].astype(np.float64)  
#         if(col%20==0):
#             print("column no : ", col, " done!")
#     end_mem = df.memory_usage().sum() / 1024**2
#     if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
#     return df

# x2 = pd.DataFrame(a)
# x2 = reduce_mem_usage(x2)

In [0]:
x2 = pd.DataFrame(a)
print(x2.shape)

summ_tfidf = x2.iloc[:len(summ_sent_mod)]
full_tfidf = x2.iloc[len(summ_sent_mod):len(summ_sent_mod)+len(full_sent_mod)]
GS_tfidf = x2.iloc[len(summ_sent_mod)+len(full_sent_mod):]

print(len(summ_docs))
print(len(full_docs))
print(len(GS_docs))

summ_tfidf['doc_id'] = summ_docs
full_tfidf['doc_id'] = full_docs
GS_tfidf['doc_id'] = full_docs

full_tfidf

In [0]:
from sklearn import preprocessing

# -----------------------------------------------------------------------------------------------------------------------------

summ_tfidf_normalized = preprocessing.normalize(summ_tfidf.drop(['doc_id'], axis=1), norm='l2')
summ_tfidf_normalized = pd.DataFrame(summ_tfidf_normalized)
summ_tfidf_normalized.columns = summ_tfidf.drop(['doc_id'], axis=1).columns
summ_tfidf_normalized['doc_id'] = np.array(summ_tfidf.doc_id)

# -----------------------------------------------------------------------------------------------------------------------------

full_tfidf_normalized = preprocessing.normalize(full_tfidf.drop(['doc_id'], axis=1), norm='l2')
full_tfidf_normalized = pd.DataFrame(full_tfidf_normalized)
full_tfidf_normalized.columns = full_tfidf.drop(['doc_id'], axis=1).columns
full_tfidf_normalized['doc_id'] = np.array(full_tfidf.doc_id)

# -----------------------------------------------------------------------------------------------------------------------------

summ_tfidf_normalized

In [0]:
A = [[2,3],[5,4]]
preprocessing.normalize(A, norm='l2')

In [0]:
cosine_vec = []

# -----------------------------------------------------------------------------------------------------------------------------

for i in set:
  tmp_full_sent = summ_tfidf_normalized[summ_tfidf_normalized.doc_id==i].drop(['doc_id'], axis=1)
  tmp_summ_sent = full_tfidf_normalized[full_tfidf_normalized.doc_id==i].drop(['doc_id'], axis=1)

  M = tmp_summ_sent.dot(np.transpose(tmp_full_sent))
  cos_doc = (np.amax(M, axis = 1))
  cosine_vec = np.append(cosine_vec, cos_doc)

# -----------------------------------------------------------------------------------------------------------------------------

len(cosine_vec)

Cosine distribution

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# -----------------------------------------------------------------------------------------------------------------------------

plt.hist(cosine_vec, bins=20)
plt.show()